In [1]:
import pandas as pd
import numpy as np 
import re

In [2]:
df = pd.read_csv("./from_skip_files/orders_11mil.csv")
item_rf = pd.read_csv("./derived_files/restaurant_profiles.csv")
rf_c = pd.read_csv('./derived_files/Cuisine_Restaraunt.csv')

In [3]:
print(df['restaurant_short_name'].nunique())
print(item_rf['short_name'].nunique())
print(df['restaurant_short_name'].str.contains("DNU").sum())
# print(df['restaurant_short_name'].str.contains("License #").sum())
def remove_after_dash(s):
    return s.split("- Li")[0].strip()

df['short_name_clean'] = df['restaurant_short_name'].apply(remove_after_dash)
df['short_name_clean'] = df['short_name_clean'].str.lower()

print(df['short_name_clean'].nunique())

list1 = df['short_name_clean'].unique()
list2 = item_rf['short_name'].unique()
list3 = rf_c['short_name'].unique()
print(len(set(list1).intersection(set(list2))))
opposite = set(list1) - set(list2)
opposite2 = set(list2) - set(list1)
print(len(opposite))
print(len(opposite2))

2474
2354
0
2382
1883
499
471


In [4]:
pd.options.display.float_format = '{:.2f}'.format
print(df['short_name_clean'].value_counts().describe())

count      2382.00
mean       4646.34
std       38116.38
min           1.00
25%          50.00
50%         524.00
75%        2434.75
max     1582092.00
Name: short_name_clean, dtype: float64


In [5]:
ethnic_or_cultural_items = [
    "Convenience",
    "Indian",
    "Italian",
    "Chinese",
    "Vietnamese",
    "Japanese",
    "Mediterranean",
    "Middle Eastern",
    "Mexican",
    "Korean",
    "Thai",
    "French",
    "African",
    "Latin American",
    "Ethiopian",
    "Caribbean",
    "Filipino",
    "Spanish",
    "German",
]

In [6]:
# Map of cuisine type to keywords that fit under that cuisine
cuisine_map = {
    "Chicken" : ["chicken"],
    "Fries" : ["fries"],
    "Beef" : ["beef"],
    "Pork" : ["pork", "bacon", "pepperoni"],
    "Rice" : ["rice"],
    "Lamb" : ["lamb"],
    "Vegetarian" : ["vegan", "vegetarian", "veggie", "beyond meat"],
    "Sandwiches & Subs" : ["sandwich", "sub", "wrap", "blt"],
    "Desserts" : ["blizzard", "ice cream", "frozen", "dessert", "chocolate", "drizzle", "desserts", "milkshake", "candy", 
                  "candies", "sundae", "oreo", "skor", "brownie", "shake", "tiramisu", "timbits", "ben and jerry", 
                  "cheesecake", "cookie"],
    "Canadian" : ["canadian", "canadien", "alberta","poutine"],
    "Fast food" : ["fast food","combo", "meal"],
    "Burgers" : ["burger", "patty", "mcdouble", "big mac", "quarter pounder"],
    "Seafood" : ["fish", "seafood", "shrimp", "crab", "lobster", "prawn", "seaweed", 
                 "salmon", "tuna", "poke", "calamari", "squid", "fish and chips"],
    "Healthy" : ["organic", "health", "protein", "salad", "fresh", "tofu",  "fruit", "water", "vegetable", "smoothie", "parfait"],
    "Pizza" : ["pizza"],
    "Breakfast & Brunch" : ["egg", "toast", "benedict", "breakfast", "brunch", "cereal", "pancake", "waffle", "hash brown"],
    "Coffee/Tea" : ["coffee", " tea ", "americano", "cappuccino", "latte", "cafe", "chai", "london fog"],
    "Alcohol" : ["beer", "wine", "liquor", "budweiser", "bud light", "spirits", "corona", "stella artois", 
                 "michelob ultra", "mike's hard", "labatt", "sauvignon", "smirnoff", "vodka", "whisky", "cognac",
                 "white claw", "pinot noir"],
    "Noodles" : ["noodle", "vermicelli"],
    "Pub food" : ["wing", "onion ring", "wedge", "mac & cheese", "mac and cheese", "gravy", "mashed potato", "breadsticks"], 
    "Indian" : ["indian", "naan", "nan", "pakora", "samosa", "masala", "aloo", "paneer", "biryani", "tandoori", "roti", "tikka"],
    "Italian" : ["italian", "pasta", "spaghetti", "penne", "fettuccini", "lasagna", "lasagne", "linguini", "ravioli", "tortellini", "meatball", "canoli"],
    "Bakery" : ["danish", "cake", "bun", "donut", "muffin","bagel", "doughnut", "pie", "scone", "cinnamon roll", "loaf"],
    "Barbecue" : ["barbecue", "bbq", "grill", "buffalo"],
    "Chinese" : ["chinese", "china", "hot pot", "wonton", "cantonese", "mein", "gyoza", "dumpling"],
    "Vietnamese" : ["vietnamese", "pho", "viet", "bun cha","ca kho to"],
    "Japanese" : ["japanese","japan", "ramen", "sashimi", "teriyaki", "katsu", "tempura", "edamame", "bento", "takoyaki"],
    "Tacos": ["taco"],
    "Sushi" : ["sushi"],
    "Mediterranean" : ["mediterranean","pita", "damascus", "greek", "greece", "briam","taramasalata", "opa"],
    "Hot Dogs & Sausages" : ["hot dogs","sausage", "weiner"],
    "Middle Eastern" : ["middle eastern","falafel", "hummus","shawarma", "baklava", "donair", "tzatziki"],
    "Convenience" : ["convenience","pre-made","grocery", "slurpee"],
    "Mexican" : ["mexican", "chilaquiles", "burrito", "nacho", "quesadilla", "queso", "taquito", "salsa"],
    "Steakhouse" : ["steakhouse", "steak"],
    "Halal" : ["halal", "zabiha"],
    "Korean" : ["korean", "kimchi", "bulgogi","bibimbap","tteokbokki","jjambbong","doenjang"],
    "Thai" : ["thai", "tom yum goong", "green curry"],
    "Soup" : ["soup"],
    "Gluten Free" : ["gluten free", "no gluten"],
    "Popcorn": ["popcorn"], 
    "Pet Food" : ["pet", "dog", "cat"],
    "Bubble Tea" : ["bubble tea", "boba", "milk tea", "taro milk"],
    "French" : ["french", "francais", "crepe", "foie gras","coq au vin","cassoulet","baguette","croissant","gougeres","cajun & creole", "creole"],
    "African" : ["african","pap en vleis", "shisa nyama","bunny chow","koshari"],
    "Latin American" : ["latin","asado","saltena","feijoada","empanada","bandeja paisa",
                        "gallo pinto","ropa vieja","mangu","encebollado", "pupusas","pepian","peruvian"],
    "Haute Cuisine" : ["haute", "high class", "expensive", "champagne"],
    "Ethiopian" : ["ethiopian","tibs","kitfo","beyainatu","fuul"],
    "Caribbean" : ["caribbean", "jamaica","barbados","bahamas"],
    "Filipino" : ["filipino","adobo","lechon","sisig","bulalo"],
    "Spanish" : ["spanish","paella valenciana","patatas bravas","gazpacho","pimientos de padron","jamon","tapas","churro"],
    "Butcher" : ["raw", "butcher","delicatessen"],
    "Kosher" : ["kosher", 'kashrut', 'jewish'],
    "German" : ["german","schnitzel","rouladen","eintopf","sauerbraten"]
}
print(len(list(cuisine_map.keys())))

54


In [7]:
grouped = df.groupby('short_name_clean')
restaurant_order_profile_list = []
def process_group(group):
    restaurant_order_profile = dict(zip(cuisine_map.keys(), [0] * 54))
    total_filtered = 0
    for index, row in group.iterrows():
        items = row['item_list'].split(", ")
        for item in items:
            lower_item = item.lower()
            cuisine_identified_item = False
            item_found = False
            for key in cuisine_map:
                for cuisine in cuisine_map[key]:
                    if cuisine in lower_item:    
                        if key in ethnic_or_cultural_items:
                            cuisine_identified_item = True  
                        restaurant_order_profile[key] += 1
                        total_filtered += 1 
                        item_found = True 
                        break
            if not cuisine_identified_item:
                restaraunt_cuisines_df = rf_c[rf_c['short_name'] == row['short_name_clean']]['cussine_list']
                if restaraunt_cuisines_df.shape[0] > 0:
                    restaraunt_cuisines_list = restaraunt_cuisines_df.str.split(", ")
                    restaraunt_cuisines = restaraunt_cuisines_list.iloc[0]
                    for cuisine in restaraunt_cuisines:
                        restaurant_order_profile[cuisine] +=1 
                        total_filtered += 1 
    # Error checking
    # print( total_filtered)
    map_total_filtered = 0
    for value in restaurant_order_profile.values():
        map_total_filtered += value
    # Normalizing
    if total_filtered == map_total_filtered and total_filtered != 0:
        for key in restaurant_order_profile:
            restaurant_order_profile[key] = restaurant_order_profile[key]/total_filtered
        restaurant_order_profile['short_name'] = group.name
        restaurant_order_profile['number_of_orders'] = group.shape[0]
        restaurant_order_profile_list.append(restaurant_order_profile)
    else:
      print("missed restauraunt: ", group.name)
      print(group)
    if len(restaurant_order_profile_list) % 300 == 0:
        print(len(restaurant_order_profile_list)/2400)

grouped.apply(process_group)

missed restauraunt:  4th avenue liquor store
                                           id  \
5496382  88f9fd7e-3630-4636-90a5-b6f6f5eaff06   

                                  customer_id             created_time  \
5496382  b8b8fc48-b307-4935-9a18-19b20e0e76b0  2022-06-14 04:05:40 UTC   

            status                         restaurant_id  \
5496382  COMPLETED  1a009a88-1cf9-4cb9-a6c9-f094794c6827   

           restaurant_short_name            accepted_time  \
5496382  4th Avenue Liquor Store  2022-06-14 04:15:09 UTC   

         price_details_delivery_fee  price_details_tip  \
5496382                        5.99                NaN   

         price_details_restaurant_subsidy  price_details_skip_subsidy  \
5496382                              0.00                        0.91   

                         accepted_at  restaurant_food_prep_minutes  \
5496382  2022-06-14 04:06:17.998 UTC                            10   

         alcohol_delivery                       item_list 

0.25
missed restauraunt:  elephant & castle
                                           id  \
1651185  8b37693b-3769-453a-92fb-f751c048c106   
1651188  5bfdadf9-ea3c-40ab-b52a-832a24756f08   

                                  customer_id             created_time  \
1651185  c3e48d9d-7031-4871-a185-a81231cc32df  2022-01-27 00:31:18 UTC   
1651188  c3e48d9d-7031-4871-a185-a81231cc32df  2022-01-27 00:11:31 UTC   

            status                         restaurant_id  \
1651185  COMPLETED  5cf415e9-2bd6-4b05-aa1d-955a36022ebe   
1651188  COMPLETED  5cf415e9-2bd6-4b05-aa1d-955a36022ebe   

        restaurant_short_name            accepted_time  \
1651185     Elephant & Castle  2022-01-27 00:46:50 UTC   
1651188     Elephant & Castle  2022-01-27 00:26:49 UTC   

         price_details_delivery_fee  price_details_tip  \
1651185                         NaN                NaN   
1651188                         NaN                NaN   

         price_details_restaurant_subsidy  price_detai

0.5
missed restauraunt:  menisa liquor
                                            id  \
11010208  11a247d7-968e-4762-9ebd-20f2b2cd16d7   

                                   customer_id             created_time  \
11010208  f5f7e6b2-3acb-4212-9c8c-b902ea01e0e9  2023-01-02 03:48:13 UTC   

             status                         restaurant_id  \
11010208  COMPLETED  7f4c8794-4f72-44f4-bb7f-d67120465be9   

         restaurant_short_name            accepted_time  \
11010208         Menisa Liquor  2023-01-02 04:07:43 UTC   

          price_details_delivery_fee  price_details_tip  \
11010208                        2.79                NaN   

          price_details_restaurant_subsidy  price_details_skip_subsidy  \
11010208                              0.00                        0.85   

                          accepted_at  restaurant_food_prep_minutes  \
11010208  2023-01-02 03:49:54.197 UTC                            19   

          alcohol_delivery             item_list short_n

0.625
missed restauraunt:  railyard brewing
                                           id  \
2934364  4f32fac9-7aa0-4646-86bf-e567b8220c0b   
3033862  2078fa0c-3cbb-44a3-8edd-e7b8f2e032d0   

                                  customer_id             created_time  \
2934364  fb7b4524-e15d-4605-9223-361b840baf74  2022-02-20 19:17:38 UTC   
3033862  6d35f508-a12a-4a40-aa61-e3f007263b80  2022-04-08 20:07:24 UTC   

            status                         restaurant_id  \
2934364  COMPLETED  825d2276-4fd4-47ac-b533-81474f0480fd   
3033862  COMPLETED  825d2276-4fd4-47ac-b533-81474f0480fd   

        restaurant_short_name            accepted_time  \
2934364      Railyard Brewing  2022-02-20 19:30:48 UTC   
3033862      Railyard Brewing  2022-04-08 20:21:58 UTC   

         price_details_delivery_fee  price_details_tip  \
2934364                        5.25                NaN   
3033862                        2.59               6.64   

         price_details_restaurant_subsidy  price_detai

missed restauraunt:  uncommon cider
                                           id  \
2574088  e92d2ffb-51a0-4a1e-831e-b2430092cab5   
2574090  a56ec1a6-16fe-4e60-96cf-2bdd184b33be   

                                  customer_id             created_time  \
2574088  cd1843b7-b8a1-48a7-a547-88d28460caab  2022-02-02 00:23:09 UTC   
2574090  cd1843b7-b8a1-48a7-a547-88d28460caab  2022-03-07 00:19:05 UTC   

            status                         restaurant_id  \
2574088  COMPLETED  8751e4a1-2054-48d4-a329-4c2fd8963cce   
2574090  COMPLETED  8751e4a1-2054-48d4-a329-4c2fd8963cce   

        restaurant_short_name            accepted_time  \
2574088        Uncommon Cider  2022-02-02 00:44:00 UTC   
2574090        Uncommon Cider  2022-03-07 00:34:00 UTC   

         price_details_delivery_fee  price_details_tip  \
2574088                        3.09               2.80   
2574090                        3.09               2.80   

         price_details_restaurant_subsidy  price_details_skip_

missed restauraunt:  wok 2 go
                                           id  \
3234731  9ddbe2e3-1323-4920-865b-3bc91601b766   
8622868  f5d91aa9-f912-46fc-acc5-e2cda039c8e0   

                                  customer_id             created_time  \
3234731  a838845b-57bf-48c1-9bed-49a3606c90fd  2022-02-15 19:20:10 UTC   
8622868  bd66f9c3-c47c-45c7-ae59-86b68bc933d0  2022-12-17 23:50:15 UTC   

            status                         restaurant_id  \
3234731  COMPLETED  920f5316-4221-4435-9cdb-18ecda4bdfff   
8622868  COMPLETED  920f5316-4221-4435-9cdb-18ecda4bdfff   

        restaurant_short_name            accepted_time  \
3234731              Wok 2 Go  2022-02-15 19:26:51 UTC   
8622868              Wok 2 Go  2022-12-17 23:57:34 UTC   

         price_details_delivery_fee  price_details_tip  \
3234731                        3.99               3.00   
8622868                         NaN                NaN   

         price_details_restaurant_subsidy  price_details_skip_subsid

""


In [8]:
# restaurant_order_profile_df = pd.DataFrame(restaurant_order_profile_list)
# restaurant_order_profile_df = pd.read_csv('restaurant_order_profile.csv')
# restaurant_order_profile_df = restaurant_order_profile_df.drop("Unnamed: 0", axis=1)
# print(df['restaurant_id'].nunique())
# print(restaurant_order_profile_df['restaurant_id'].nunique())

In [9]:
restaurant_order_profile_df = pd.DataFrame(restaurant_order_profile_list)
restaurant_order_profile_df = restaurant_order_profile_df.reindex(columns=['short_name', 'number_of_orders'] + list(restaurant_order_profile_df.columns.drop(['short_name', 'number_of_orders'])))
cols = restaurant_order_profile_df.columns.tolist()
cols = cols[:-54] + sorted(cols[-54:])
restaurant_order_profile_df = restaurant_order_profile_df.reindex(columns=cols)
print(restaurant_order_profile_df)

                     short_name  number_of_orders  African  Alcohol  Bakery  \
0                         1 pot                12     0.00     0.00    0.08   
1          1600 world bier haus              2094     0.00     0.00    0.05   
2      17th ave liquor boutique              8940     0.00     0.65    0.00   
3                  1886 buffalo               334     0.00     0.00    0.06   
4               2 claw burritos               129     0.00     0.00    0.00   
...                         ...               ...      ...      ...     ...   
2357  zaika the indian delicacy              5260     0.00     0.00    0.00   
2358       zaks eatery & market                 2     0.00     0.00    0.00   
2359                 zcrew cafe              1547     0.00     0.00    0.01   
2360        zimidara restaurant               738     0.00     0.00    0.00   
2361                      zushi                75     0.00     0.00    0.07   

      Barbecue  Beef  Breakfast & Brunch  Bubble Te

In [10]:
restaurant_order_profile_df.to_csv('./derived_files/restaurant_order_profile.csv', index=False)

In [11]:
def calculate_weight(num_orders, upper_bound):
    weight = 0
    if(num_orders >= 50 and num_orders <= 500):
        weight = (num_orders**(0.06524387406))-1 # This num comes from making weight 0.5 at 500 with exponents
        # ln(1.5)/ln(500)
    elif(num_orders > 500 and num_orders <= 100):
        weight = 0.001*(num_orders) # Slope that becomes 1.0 at 600 orders
    elif(num_orders < 50):
        weight = 0
    else:
        weight = 1
    # multiply weights by upper bound    
    if(weight>upper_bound):
        weight = upper_bound
    if(weight < 0):
        weight = 0
    return weight

In [12]:
def combine_order_and_item_profile(order, item, weight):
    new_vector = (weight)*(order.values)+(1-weight)*(item.values)
    return new_vector

In [13]:
print(item_rf.shape)
print(restaurant_order_profile_df.shape)

(2354, 55)
(2362, 56)


In [14]:
# Find restaraunt overlap between orders and items
print(item_rf.shape)
print(restaurant_order_profile_df.shape)
overlap = restaurant_order_profile_df['short_name'].isin(item_rf['short_name'])
ro_filtered = restaurant_order_profile_df[overlap]
print(ro_filtered.shape)

# For restaraunts overlap between orders and items 
rows = []
upper_bound_order_weighting = 0.8
for index, ord_row in ro_filtered.iterrows():
    num_orders = ord_row['number_of_orders']
    r_id = ord_row['short_name']
    # Get weight of combination
    weight = calculate_weight(num_orders, upper_bound_order_weighting)
#     print(num_orders, " -- ", weight)
    item_row = item_rf[item_rf['short_name'] == r_id]
    item_row = item_row.squeeze()
    item_row = item_row.drop('short_name')
    ord_row = ord_row.drop("number_of_orders")
    ord_row = ord_row.drop('short_name')
    new_vector = combine_order_and_item_profile(ord_row, item_row, weight)
    new_vector = np.insert(new_vector, 0, r_id)
    rows.append(new_vector)

(2354, 55)
(2362, 56)
(1878, 56)


In [15]:
column_names = item_rf.columns.to_list()
combined_df = pd.DataFrame(rows, columns=column_names)
print(combined_df.shape)

(1878, 55)


In [16]:
# Find missing restaraunts (restaurants not found in any orders)
difference = ~item_rf['short_name'].isin(restaurant_order_profile_df['short_name'])
no_order_restauraunts = item_rf[difference]

In [17]:
result_df = pd.concat([combined_df, no_order_restauraunts])
print(result_df.shape)
print(result_df['short_name'].nunique())
# Make sure each row adds to 1 
for index, row in result_df.iterrows():
    if row[1:].sum() > 1.05 or row[1:].sum() < 0.95:
      print(row[1:].sum())
      print("Something is wrong at index: ", index)


(2354, 55)
2354


In [18]:
result_df.to_csv('./derived_files/combined_restaurant_order_profile.csv', index=False)